In [261]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyarrow.parquet as pq

In [224]:
raw_data = pd.read_csv('./dataset/heart_disease_uci.csv')

In [225]:
raw_data.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num,created_timestamp
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0,2025-01-01 00:00:00.000000
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2,2025-01-01 00:00:00.000000
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1,2025-01-01 00:00:00.000000
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0,2025-01-01 00:00:00.000000
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0,2025-01-01 00:00:00.000000


In [226]:
raw_data_with_create_timestamp = raw_data.copy()  
raw_data_with_create_timestamp['created_timestamp'] = pd.to_datetime('2025-01-01 00:00:00')

In [227]:
raw_data_with_create_timestamp.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num,created_timestamp
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0,2025-01-01
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2,2025-01-01
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1,2025-01-01
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0,2025-01-01
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0,2025-01-01


In [228]:
def generate_random_row(record_id):
    age = np.random.randint(29, 77)  
    sex = np.random.choice(['Male', 'Female'])  
    dataset = np.random.choice(['Cleveland', 'Hungary', 'Switzerland', 'VA Long Beach'])  
    cp = np.random.choice(['typical angina', 'atypical angina', 'non-anginal', 'asymptomatic'])
    trestbps = np.random.randint(94, 200)  
    chol = np.random.randint(126, 564)  
    fbs = np.random.choice(['False', 'True'])  
    restecg = np.random.choice(['normal', 'stt abnormality', 'lv hypertrophy'])
    thalach = np.random.randint(71, 202)  
    exang = np.random.choice([0, 1])  
    oldpeak = np.random.uniform(0, 6)  
    slope = np.random.choice([0, 1, 2])  
    ca = np.random.choice([0, 1, 2, 3, 4])  
    thal = np.random.choice(['normal', 'fixed defect', 'reversible defect'])
    target = np.random.choice([0, 1])  
    created_timestamp = datetime.now()  # Bierzemy aktualny timestamp
    return [record_id, age, sex, dataset, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal, target, created_timestamp]

# Dodajemy 10 nowych rekordów z aktualnym timestampem i id
new_records = []
for record_id in range(len(raw_data)+1, len(raw_data)+11):  # Zaczynamy od id po ostatnim wierszu
    new_row = generate_random_row(record_id)  
    new_records.append(new_row)

In [229]:
new_df = pd.DataFrame(new_records, columns=raw_data_with_create_timestamp.columns)

In [255]:
raw_data_with_create_timestamp = raw_data_with_create_timestamp.astype(str)
new_df = new_df.astype(str)

In [256]:
combined_data = pd.concat([raw_data_with_create_timestamp, new_df], ignore_index=True)


In [ ]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
file_name = f'heart_disease_uci_delta_load_{timestamp}.parquet'

In [258]:
combined_data.to_parquet(f'./dataset/delta/{file_name}', index=False)

test

In [263]:
file_path = './dataset/delta/heart_disease_uci_delta_load_2025-02-04_22-27-32.parquet'

In [267]:
table = pq.read_table(file_path)
df = table.to_pandas()

In [268]:
display(df)

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num,created_timestamp
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0,2025-01-01
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2,2025-01-01
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1,2025-01-01
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0,2025-01-01
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0,2025-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,936,50,Male,Cleveland,atypical angina,183,322,False,normal,144,0,3.6977411100903073,0,0,reversible defect,0,2025-02-04 22:24:27.955848
936,937,67,Female,Cleveland,non-anginal,118,486,True,stt abnormality,144,0,3.11312802684256,1,3,reversible defect,1,2025-02-04 22:24:27.955848
937,938,51,Male,Switzerland,typical angina,138,447,True,normal,83,1,3.8536949452814078,1,4,normal,0,2025-02-04 22:24:27.955848
938,939,29,Female,Cleveland,non-anginal,124,260,True,lv hypertrophy,82,0,2.5173827331933705,2,1,normal,0,2025-02-04 22:24:27.956845
